In [1]:
import pandas as pd 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('../dataset/Udemy_Cleaned.csv')

In [3]:
df.head()

,id,course_id,rate,date,display_name,comment,title,is_paid,price,headline,...,published_time,last_update_date,category,subcategory,topic,language,course_url,instructor_name,instructor_url,image
0,9388140,625204,5.0,2017-09-08T02:15:37-07:00,John Douglas,"examples are well thought through, the lessons...",The Web Developer Bootcamp 2022,True,199.99,COMPLETELY REDONE - The only course you need t...,...,2015-11-02 21:13:27+00:00,2022-07-01,Development,Web Development,Web Development,English,/course/the-web-developer-bootcamp/,Colt Steele,/user/coltsteele/,https://img-c.udemycdn.com/course/750x422/6252...
1,125232566,821278,4.0,2022-10-04T01:40:55-07:00,Tanya,Helps in increasing knowledge and run through ...,"INTERVIEW TRAINING: Ask Better Questions, Hire...",True,19.99,If you want to improve your hiring accuracy or...,...,2016-04-21 15:00:43+00:00,2017-01-31,Business,Management,Interviewing Skills,English,/course/advanced-interviewer-training/,James Colino,/user/james-colino/,https://img-b.udemycdn.com/course/750x422/8212...
2,57899017,567828,5.0,2020-06-09T05:51:08-07:00,Rohan,Yes. I don't feel overwhelmed about learning h...,2022 Complete Python Bootcamp From Zero to Her...,True,199.99,Learn Python like a Professional Start from t...,...,2015-10-12 21:42:53+00:00,2021-03-14,Development,Programming Languages,Python,English,/course/complete-python-bootcamp/,Jose Portilla,/user/joseportilla/,https://img-c.udemycdn.com/course/750x422/5678...
3,43107550,1352468,4.0,2020-01-06T06:01:41-08:00,Abhishek,Overall course was good except last two lectur...,Master Microservices with Spring Boot and Spri...,True,199.99,Java Spring Boot Microservices 5-in-1 - Spring...,...,2017-09-18 23:32:14+00:00,2022-10-03,Development,Web Development,Microservices,English,/course/microservices-with-spring-boot-and-spr...,in28Minutes Official,/user/in28minutes/,https://img-c.udemycdn.com/course/750x422/1352...
4,23581421,1915496,5.0,2018-11-25T19:35:03-08:00,Bryan T,I love the speech delivery speed and articulat...,Facebook Marketing - Build Facebook Messenger...,True,199.99,"Facebook marketing & Facebook ads: Design, dev...",...,2018-09-18 18:14:25+00:00,2019-06-05,Marketing,Digital Marketing,Facebook Marketing,English,/course/facebook-messenger-chatbot-marketing-t...,Isaac Rudansky,/user/isaac-rudansky/,https://img-c.udemycdn.com/course/750x422/1915...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263591 entries, 0 to 263590
Data columns (total 26 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  263591 non-null  int64  
 1   course_id           263591 non-null  int64  
 2   rate                263591 non-null  float64
 3   date                263591 non-null  object 
 4   display_name        263591 non-null  object 
 5   comment             263591 non-null  object 
 6   title               263591 non-null  object 
 7   is_paid             263591 non-null  bool   
 8   price               263591 non-null  float64
 9   headline            263591 non-null  object 
 10  num_subscribers     263591 non-null  int64  
 11  avg_rating          263591 non-null  float64
 12  num_reviews         263591 non-null  int64  
 13  num_comments        263591 non-null  int64  
 14  num_lectures        263591 non-null  int64  
 15  content_length_min  263591 non-nul

In [5]:
# get sample data; take 5%
df = df.sample(frac=0.5)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 131796 entries, 188292 to 235027
Data columns (total 26 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  131796 non-null  int64  
 1   course_id           131796 non-null  int64  
 2   rate                131796 non-null  float64
 3   date                131796 non-null  object 
 4   display_name        131796 non-null  object 
 5   comment             131796 non-null  object 
 6   title               131796 non-null  object 
 7   is_paid             131796 non-null  bool   
 8   price               131796 non-null  float64
 9   headline            131796 non-null  object 
 10  num_subscribers     131796 non-null  int64  
 11  avg_rating          131796 non-null  float64
 12  num_reviews         131796 non-null  int64  
 13  num_comments        131796 non-null  int64  
 14  num_lectures        131796 non-null  int64  
 15  content_length_min  131796 non-nul

In [7]:
unique_df = df.drop_duplicates(subset='title').reset_index(drop=True)
unique_df = unique_df[['course_id', 'title', 'category', 'subcategory', 'topic']]

In [8]:
unique_df.head()

,course_id,title,category,subcategory,topic
0,4106838,Adobe Premiere Pro CC Video Editing Course for...,Photography & Video,Video Design,Video Editing
1,671544,Ultimate Google Ads Training 2021: Profit with...,Marketing,Paid Advertising,Google Ads (Adwords)
2,1317814,Mastering Photographic Exposure - Learn the Fu...,Photography & Video,Photography Tools,Photography
3,677348,Supply Chain Fundamentals: Understanding the ...,Business,Operations,Supply Chain
4,2097796,Online Coaching Business: Health & Life Coach ...,Business,Entrepreneurship,Career Coaching


In [9]:
unique_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22885 entries, 0 to 22884
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course_id    22885 non-null  int64 
 1   title        22885 non-null  object
 2   category     22885 non-null  object
 3   subcategory  22885 non-null  object
 4   topic        22885 non-null  object
dtypes: int64(1), object(4)
memory usage: 894.1+ KB


In [35]:
# Create a TfidfVectorizer object to transform the courses subcategory into a Tf-idf representation
def compute_similarity(df):
    sub = df['subcategory'].str.split('|').explode()
    
    tfidf = TfidfVectorizer(ngram_range=(1,2))
    tfidf_matrix = tfidf.fit_transform(sub)
    
    similarity = cosine_similarity(tfidf_matrix)
    return similarity

In [36]:
similarity_df = compute_similarity(unique_df)

In [37]:
similarity_df

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.13585527, ..., 0.13585527, 0.        ,
        0.        ],
       [0.        , 0.13585527, 1.        , ..., 1.        , 0.        ,
        0.20078843],
       ...,
       [0.        , 0.13585527, 1.        , ..., 1.        , 0.        ,
        0.20078843],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.20078843, ..., 0.20078843, 0.        ,
        1.        ]])

In [38]:
unique_df[unique_df['title'] == 'Python & Machine Learning for Financial Analysis']

,course_id,title,category,subcategory,topic
1082,3428726,Python & Machine Learning for Financial Analysis,Development,Data Science,Financial Analysis


In [40]:
similarity_df[1082]

array([0.        , 0.13585527, 1.        , ..., 1.        , 0.        ,
       0.20078843])

In [41]:
# Function to get the recommended course
def recommendations(title, top_n=10):
    # Find the index of the courses with the given title
    idx = unique_df[unique_df['title'] == title].index[0]

    # Get the cosine similarity scores for the courses
    similarity_scores = list(enumerate(similarity_df[idx]))

    # Sort the similarity scores in descending order
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the top_n courses indices
    courses_indices = [i[0] for i in similarity_scores[1:top_n+1]]

    # Return the top_n most similar courses
    return unique_df['title'].iloc[courses_indices]

In [42]:
course = 'Python & Machine Learning for Financial Analysis'

recommendations(course)

19     Ultimate Kubernetes Fast-Track Beginner to Adv...
29     A Beginners Guide to Exadata Patching for Orac...
49      UiPath - The Complete RPA Training Course (2022)
80                           Microsoft Flow Crash Course
109    Spring MVC For Beginners - Build Java Web App ...
112                    Java Debugging With IntelliJ IDEA
127    Master Spring Microservices with Spring Boot &...
137                  The Complete C Programming Tutorial
148    RPA - Process Automation using UIPATH - Beginn...
175    Introduction to AWS - EC2 Deployment Fundamentals
Name: title, dtype: object

In [43]:
course = 'Python & Machine Learning for Financial Analysis'


sim = pd.DataFrame(similarity_df, index=unique_df['title'], columns=unique_df['title'])

def get_top_n(sim, title):
    
    course_index = sim.index.get_loc(title)
    
    top_10 = sim.iloc[course_index].sort_values(ascending=False)[1:11]
    return top_10.index.tolist()

In [45]:
get_top_n(sim, course)

['Build 75 Powerful Data Science & Machine Learning Projects',
 'Data Science A-Z : Machine Learning with Python & R',
 'Python for Statistical Analysis',
 'Natural Language Processing: NLP With Transformers in Python',
 'Tensorflow 2.0: Deep Learning and Artificial Intelligence',
 'Architecting Big Data Solutions',
 'Probability and Statistics for Business and Data Science',
 'Conversational AI (Chatbots) and NLP using Javascript',
 'Unsupervised Machine Learning Hidden Markov Models in Python',
 'Apache Spark 2.0 + Java : DO Big Data Analytics & ML']

In [15]:
titles = unique_df['title'].tolist()
subcategory = unique_df['subcategory'].str.split("|").tolist()

In [16]:
def create_bow(subcategory):
    bow = {}
    for sub in subcategory:
        bow[sub] = 1
    return bow

In [17]:
bags_of_words = [create_bow(sub) for sub in subcategory]

In [18]:
sub_df = pd.DataFrame(bags_of_words, index=titles).fillna(0)

In [19]:
similarity = cosine_similarity(sub_df)

In [20]:
# Create a dataframe with the cosine similarity scores
sim = pd.DataFrame(similarity, index=sub_df.index, columns=sub_df.index)

In [21]:
get_top_n(sim, course)

['Apache Spark Streaming 3 with Scala | Rock the JVM',
 'Deep Learning: Visual Exploration',
 'Natural Language Processing with Deep Learning in Python',
 'Intro to Big Data, Data Science and Artificial Intelligence',
 'Artificial Intelligence III - Deep Learning in Java',
 'R Programming – Hacks and Automation',
 'Automated Machine Learning for Beginners (Google & Apple)',
 'Modern Natural Language Processing in Python',
 'Species Distribution Models with GIS & Machine Learning in R',
 'Machine Learning 2022: Complete Maths for Machine Learning']

In [10]:
subcategory_split = unique_df['subcategory']

In [11]:
# Using get_dummies to create a binary feature matrix
subcategory_dummies = pd.get_dummies(subcategory_split).groupby(level=0).sum()
subcategory_dummies

,3D & Animation,Accounting & Bookkeeping,Affiliate Marketing,Apple,Architectural Design,Arts & Crafts,Beauty & Makeup,Branding,Business Analytics & Intelligence,Business Law,...,Taxes,Test Prep,Travel,User Experience Design,Video & Mobile Marketing,Video Design,Vocal,Web Design,Web Development,Yoga
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22880,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22881,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22882,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22883,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
subcategory_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22885 entries, 0 to 22884
Columns: 112 entries, 3D & Animation to Yoga
dtypes: int64(112)
memory usage: 19.6 MB


In [13]:
# Compute the cosine similarity matrix
similarity = cosine_similarity(subcategory_dummies)
similarity

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [14]:
# Function to get the recommended course
def recommendations(title, top_n=10):
    # Find the index of the courses with the given title
    idx = unique_df[unique_df['title'] == title].index[0]

    # Get the cosine similarity scores for the courses
    similarity_scores = list(enumerate(similarity[idx]))

    # Sort the similarity scores in descending order
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the top_n courses indices
    courses_indices = [i[0] for i in similarity_scores[1:top_n+1]]

    # Return the top_n most similar courses
    return unique_df['title'].iloc[courses_indices]

In [15]:
course = 'Python & Machine Learning for Financial Analysis'

In [16]:
unique_df[unique_df['title'] == course]

,course_id,title,category,subcategory,topic
3655,3428726,Python & Machine Learning for Financial Analysis,Development,Data Science,Financial Analysis


In [17]:
recommendations(course)

122                 Data Analysis with Pandas and Python
141    The Data Science Course 2022: Complete Data Sc...
160                       Modern Deep Learning in Python
169    Deep Learning Prerequisites: Linear Regression...
306    Statistics for Data Science and Business Analysis
307    Machine Learning A-Z™: Hands-On Python & R In ...
316        NLP - Natural Language Processing with Python
394    Deep Learning A-Z™: Hands-On Artificial Neural...
395     Text Mining and Natural Language Processing in R
428    Azure Cloud Azure Databricks Apache Spark Mach...
Name: title, dtype: object